A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [31]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [32]:
# 삼성전자
sample_code = '005930'

In [33]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

INDEX_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'



In [34]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [35]:
per

8.15

In [36]:
snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
market_cap = snap_tree.xpath('//*[@id="svdMainGrid1"]/table/tbody/tr[5]/td[1]')[0].text
market_cap = float(market_cap.replace(',',''))

In [37]:
market_cap

3922147.0

In [38]:
# object -> 데이터 덩어리(추상화)

index_url = INDEX_URL.format(sample_code)
index_content = requests.get(index_url).content # 문자열 binary
index_tree = html.fromstring(index_content) # 객체(object)
ev = index_tree.xpath('//*[@id="p_grid1_14"]/td[5]')[0].text
ev = float(ev)


psr = index_tree.xpath('//*[@id="p_grid1_11"]/td[5]')[0].text
psr = float(psr)

In [39]:
ev

4.62

In [40]:
psr

1.24

In [41]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [42]:
debt_ratio

26.4

In [43]:
# gross profit

fs_url = FS_URL.format(sample_code)
fs_content = requests.get(fs_url).content
fs_tree = html.fromstring(fs_content)
gross_profit = fs_tree.xpath('//*[@id="divSonikY"]/table/tbody/tr[3]/td[4]')[0].text
gross_profit = float(gross_profit.replace(',',''))

asset = fs_tree.xpath('//*[@id="divDaechaY"]/table/tbody/tr[1]/td[4]')[0].text
asset = float(asset.replace(',',''))


In [44]:
gross_profit

1121896.0

In [45]:
asset

4484245.0

In [46]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [47]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [48]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [49]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [50]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [51]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [52]:
print(stock_list_df.shape)

(2461, 9)


In [53]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [54]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [55]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [56]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [57]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [58]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [59]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [60]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [85]:
def FinanceInfoCrawler2(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            INDEX_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
            FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get market_cap

            market_cap = snap_tree.xpath('//*[@id="svdMainGrid1"]/table/tbody/tr[5]/td[1]')[0].text
            market_cap = float(market_cap.replace(',',''))

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)

            # Get PSR, EV
            index_url = INDEX_URL.format(code)
            index_content = requests.get(index_url).content # 문자열 binary
            index_tree = html.fromstring(index_content) # 객체(object)
            ev = index_tree.xpath('//*[@id="p_grid1_14"]/td[5]')[0].text
            ev = float(ev)


            psr = index_tree.xpath('//*[@id="p_grid1_11"]/td[5]')[0].text
            psr = float(psr)

            #Get Gross Profit, Asset

            # gross profit

            fs_url = FS_URL.format(code)
            fs_content = requests.get(fs_url).content
            fs_tree = html.fromstring(fs_content)
            gross_profit = fs_tree.xpath('//*[@id="divSonikY"]/table/tbody/tr[3]/td[4]')[0].text
            gross_profit = float(gross_profit.replace(',',''))

            asset = fs_tree.xpath('//*[@id="divDaechaY"]/table/tbody/tr[1]/td[4]')[0].text
            asset = float(asset.replace(',',''))
            
            result_dict[company_name] = [code,company_name,market_cap, per, psr,ev,gross_profit, asset, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['code','Name','Market_Cap', 'PER','PSR','EV_EBITDA','Gross_Profit','Asset', 'Debt_ratio']
    
    # return result_df, error_codes
    return result_df
    

In [86]:
crawling_result_df2 = FinanceInfoCrawler2(code_list[:50],stock_list_df)
print(crawling_result_df2.shape)
crawling_result_df2.head()

100%|██████████| 50/50 [01:30<00:00,  1.80s/it]

(37, 9)


,code,Name,Market_Cap,PER,PSR,EV_EBITDA,Gross_Profit,Asset,Debt_ratio
SK오션플랜트,100090,SK오션플랜트,10381.0,38.11,1.26,12.9,1014.0,12200.0,132.0
바이오노트,377740,바이오노트,6302.0,1.89,1.68,2.9,3407.0,17570.0,9.8
유니드비티플러스,446070,유니드비티플러스,1000.0,104.18,3.05,29.33,46.0,2136.0,11.2
LX세미콘,108320,LX세미콘,17533.0,7.5,0.55,3.42,6757.0,12896.0,35.7
수산인더스트리,126720,수산인더스트리,3171.0,6.3,0.86,5.13,740.0,5772.0,24.7


In [87]:
copy_df = crawling_result_df2.copy()

In [88]:
copy_df['GP_A'] = copy_df['Gross_Profit'] / copy_df['Asset']

In [89]:
copy_df = copy_df[['code', 'EV_EBITDA', 'GP_A']]

In [90]:
copy_df

,code,EV_EBITDA,GP_A
SK오션플랜트,100090,12.9,0.083115
바이오노트,377740,2.9,0.19391
유니드비티플러스,446070,29.33,0.021536
LX세미콘,108320,3.42,0.523961
수산인더스트리,126720,5.13,0.128205
LG에너지솔루션,373220,34.66,0.112025
SK스퀘어,402340,10.83,0.203546
케이카,381970,7.6,0.374251
일진하이솔루스,271940,131.97,0.068613
아주스틸,139990,25.21,0.058011


In [91]:
# |-----------|---------------|-----------|
# |    30%.   |       40%.    |.    30%.   |
# 0.         N_1             N_2           N_3

In [92]:
high_range = int(len(copy_df) * 0.3)
middle_range = int(len(copy_df) * 0.7)

In [93]:
print(high_range, middle_range)

11 25


In [94]:
# EV_EBITDA
# 오름차순 정렬
# 30 : 40 : 30
#  3    2    1
copy_df.loc[copy_df['code'].isin(copy_df.sort_values(by="EV_EBITDA", ascending=True).iloc[:high_range]['code']), 'EV_EBITDA_Score'] = 3                   
copy_df.loc[copy_df['code'].isin(copy_df.sort_values(by='EV_EBITDA', ascending=True).iloc[high_range:middle_range]['code']), 'EV_EBITDA_Score'] = 2
copy_df.loc[copy_df['code'].isin(copy_df.sort_values(by='EV_EBITDA', ascending=True).iloc[middle_range:]['code']), 'EV_EBITDA_Score'] = 1 


In [95]:
copy_df['EV_EBITDA_Score'].value_counts()

2.0    14
1.0    12
3.0    11
Name: EV_EBITDA_Score, dtype: int64

In [96]:
copy_df.head()

,code,EV_EBITDA,GP_A,EV_EBITDA_Score
SK오션플랜트,100090,12.9,0.083115,1.0
바이오노트,377740,2.9,0.19391,3.0
유니드비티플러스,446070,29.33,0.021536,1.0
LX세미콘,108320,3.42,0.523961,3.0
수산인더스트리,126720,5.13,0.128205,3.0


In [97]:
# GP_A
# 내림차순 정렬
# 30 : 40 : 30
# 3    2     1
# GP_A_Score 

copy_df.loc[copy_df['code'].isin(copy_df.sort_values(by='GP_A', ascending=False).iloc[:high_range]['code']), 'GP_A_Score'] = 3
copy_df.loc[copy_df['code'].isin(copy_df.sort_values(by='GP_A', ascending=False).iloc[high_range:middle_range]['code']), 'GP_A_Score'] = 2
copy_df.loc[copy_df['code'].isin(copy_df.sort_values(by='GP_A', ascending=False).iloc[middle_range:]['code']), 'GP_A_Score'] = 1       



In [98]:
copy_df

,code,EV_EBITDA,GP_A,EV_EBITDA_Score,GP_A_Score
SK오션플랜트,100090,12.9,0.083115,1.0,1.0
바이오노트,377740,2.9,0.19391,3.0,2.0
유니드비티플러스,446070,29.33,0.021536,1.0,1.0
LX세미콘,108320,3.42,0.523961,3.0,3.0
수산인더스트리,126720,5.13,0.128205,3.0,2.0
LG에너지솔루션,373220,34.66,0.112025,1.0,1.0
SK스퀘어,402340,10.83,0.203546,2.0,2.0
케이카,381970,7.6,0.374251,2.0,3.0
일진하이솔루스,271940,131.97,0.068613,1.0,1.0
아주스틸,139990,25.21,0.058011,1.0,1.0


In [99]:
# 총합 구하기
# Total_Score
copy_df['Total_Score'] = copy_df['EV_EBITDA_Score'] + copy_df['GP_A_Score']

In [100]:
copy_df

,code,EV_EBITDA,GP_A,EV_EBITDA_Score,GP_A_Score,Total_Score
SK오션플랜트,100090,12.9,0.083115,1.0,1.0,2.0
바이오노트,377740,2.9,0.19391,3.0,2.0,5.0
유니드비티플러스,446070,29.33,0.021536,1.0,1.0,2.0
LX세미콘,108320,3.42,0.523961,3.0,3.0,6.0
수산인더스트리,126720,5.13,0.128205,3.0,2.0,5.0
LG에너지솔루션,373220,34.66,0.112025,1.0,1.0,2.0
SK스퀘어,402340,10.83,0.203546,2.0,2.0,4.0
케이카,381970,7.6,0.374251,2.0,3.0,5.0
일진하이솔루스,271940,131.97,0.068613,1.0,1.0,2.0
아주스틸,139990,25.21,0.058011,1.0,1.0,2.0


In [101]:
# 총점 높은 순으로 정렬
copy_df.sort_values(by='Total_Score', ascending=False)

,code,EV_EBITDA,GP_A,EV_EBITDA_Score,GP_A_Score,Total_Score
에스디바이오센서,137310,2.65,0.427052,3.0,3.0,6.0
대덕전자,353200,2.83,0.249119,3.0,3.0,6.0
LX세미콘,108320,3.42,0.523961,3.0,3.0,6.0
롯데렌탈,089860,3.62,0.395383,3.0,3.0,6.0
자이에스앤디,317400,2.05,0.152302,3.0,2.0,5.0
에이플러스에셋,244920,7.45,0.393281,2.0,3.0,5.0
바이오노트,377740,2.9,0.19391,3.0,2.0,5.0
F&F,383220,9.41,0.812763,2.0,3.0,5.0
LX홀딩스,383800,4.02,0.12159,3.0,2.0,5.0
HD현대에너지솔루션,322000,5.45,0.283854,2.0,3.0,5.0


In [102]:
# 총점 Max 종목만 필터링
final_result_df = copy_df[copy_df['Total_Score'] == 6].sort_values(by='GP_A', ascending=False).head(30)

In [103]:
final_result_df

,code,EV_EBITDA,GP_A,EV_EBITDA_Score,GP_A_Score,Total_Score
LX세미콘,108320,3.42,0.523961,3.0,3.0,6.0
에스디바이오센서,137310,2.65,0.427052,3.0,3.0,6.0
롯데렌탈,089860,3.62,0.395383,3.0,3.0,6.0
대덕전자,353200,2.83,0.249119,3.0,3.0,6.0


In [104]:
final_result_df.to_csv('Magic_Formula.csv')

In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
